In [1]:
#모의고사2회_제2유형


In [2]:
# 데이터셋 : 'https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv'
# 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상태를 status(1:파킨슨병 진단,0:파킨슨병 아님)로 
# 추가한 테이블이다.

In [3]:
# 데이터셋을 이용하여 파킨슨병을 예측하는 모델을 로지스틱 회귀모형을 적용하여 생성하고, 이 때 파킨슨병을 예측하는데
# 영향을 미치는 변수를 중요한 순서대로 3개 선정하시오.
# 이 모델에서 파킨슨병으로 진단하는 기준(threshold, 또는 cutoff)을 0.5로 했을때와 0.8로 했을때의
# F1-SCORE를 비교하고 해석하시오. (단, 다음 조건을 지켜서 물음에 답하시오.)


# <분석조건> 
# 필요없는 컬럼인 name 제거
# 데이터 정규화는 min-max 스케일러 사용
# 로지스틱 회귀를 위해 상수항 추가
# status는 카테고리 타입으로 변환
# 트레이닝셋과 테스트 셋의 비율은 9:1
# 모델은 로지스틱 회귀분석 사용
# 모델의 최적화 방법론으로 bgfs 사용

In [4]:
import numpy as np
import pandas as pd

In [5]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from sklearn import metrics
from sklearn.metrics import f1_score

In [6]:
dat = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv')

In [7]:
dat.head(10)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
5,phon_R01_S01_6,120.552,131.162,113.787,0.00968,0.00008,0.00463,0.00750,0.01388,0.04701,...,0.06985,0.01222,21.378,1,0.415564,0.825069,-4.242867,0.299111,2.187560,0.357775
6,phon_R01_S02_1,120.267,137.244,114.820,0.00333,0.00003,0.00155,0.00202,0.00466,0.01608,...,0.02337,0.00607,24.886,1,0.596040,0.764112,-5.634322,0.257682,1.854785,0.211756
7,phon_R01_S02_2,107.332,113.840,104.315,0.00290,0.00003,0.00144,0.00182,0.00431,0.01567,...,0.02487,0.00344,26.892,1,0.637420,0.763262,-6.167603,0.183721,2.064693,0.163755
8,phon_R01_S02_3,95.730,132.068,91.754,0.00551,0.00006,0.00293,0.00332,0.00880,0.02093,...,0.03218,0.01070,21.812,1,0.615551,0.773587,-5.498678,0.327769,2.322511,0.231571
9,phon_R01_S02_4,95.056,120.103,91.226,0.00532,0.00006,0.00268,0.00332,0.00803,0.02838,...,0.04324,0.01022,21.862,1,0.547037,0.798463,-5.011879,0.325996,2.432792,0.271362


In [8]:
dat_preprocessing = dat.drop(['name'], axis=1, inplace=False)

In [9]:
dat_preprocessing_norm = preprocessing.minmax_scale(dat_preprocessing)
dat_processed = pd.DataFrame(dat_preprocessing_norm)
dat_processed.columns = dat_preprocessing.columns

In [10]:
dat_processed = sm.add_constant(dat_processed, has_constant='add')
dat_processed.head(10)

,const,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,1.0,0.184308,0.112592,0.054815,0.195680,0.249012,0.145472,0.247588,0.145288,0.312215,...,0.332584,0.068307,0.511745,1.0,0.369155,0.960148,0.569875,0.585765,0.390661,0.497310
1,1.0,0.198327,0.094930,0.278323,0.254130,0.288538,0.191233,0.323687,0.191042,0.472887,...,0.516048,0.059331,0.432577,1.0,0.470830,0.977024,0.703277,0.741337,0.473145,0.671326
2,1.0,0.165039,0.059128,0.265288,0.280178,0.328063,0.229287,0.369239,0.229411,0.390634,...,0.443317,0.039596,0.496220,1.0,0.404416,1.000000,0.636745,0.686371,0.408819,0.596682
3,1.0,0.165004,0.072927,0.264200,0.263342,0.328063,0.209056,0.324759,0.208862,0.414278,...,0.475478,0.040997,0.495936,1.0,0.416255,0.975885,0.695627,0.738089,0.436977,0.671949
4,1.0,0.161150,0.080909,0.260107,0.354511,0.407115,0.282755,0.437299,0.282870,0.499452,...,0.584542,0.054174,0.455499,1.0,0.375159,0.992813,0.762472,0.513798,0.404336,0.757611
5,1.0,0.187568,0.059232,0.278139,0.254130,0.288538,0.190270,0.352626,0.190079,0.342067,...,0.360829,0.036827,0.525766,1.0,0.370978,0.999128,0.672961,0.659218,0.339999,0.648753
6,1.0,0.185909,0.071647,0.284086,0.052414,0.090909,0.041908,0.058950,0.042061,0.059704,...,0.062460,0.017252,0.668333,1.0,0.792079,0.756277,0.421385,0.565955,0.191959,0.346328
7,1.0,0.110606,0.023873,0.223606,0.038755,0.090909,0.036609,0.048232,0.036442,0.055961,...,0.072089,0.008881,0.749858,1.0,0.888630,0.752890,0.324968,0.399458,0.285340,0.246912
8,1.0,0.043063,0.061082,0.151289,0.121665,0.209486,0.108382,0.128617,0.108525,0.103980,...,0.119014,0.031989,0.543404,1.0,0.837604,0.794025,0.445910,0.723731,0.400034,0.387368
9,1.0,0.039139,0.036658,0.148249,0.115629,0.209486,0.096339,0.128617,0.096163,0.171992,...,0.190012,0.030461,0.545436,1.0,0.677741,0.893130,0.533923,0.719740,0.449094,0.469780


In [11]:
feature_columns = list(dat_processed.columns.difference(["status"]))

X = dat_processed[feature_columns]
y = dat_processed['status'].astype('category')

In [12]:
train_x, test_x, train_y, test_y = train_test_split(X, y, stratify=y, test_size=0.1, random_state=2017010500)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((175, 23), (20, 23), (175,), (20,))

In [13]:
model = sm.Logit(train_y, train_x)

In [15]:
results = model.fit(method='bfgs', maxiter=1000)

Optimization terminated successfully.
         Current function value: 0.224375
         Iterations: 315
         Function evaluations: 316
         Gradient evaluations: 316


In [16]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 status   No. Observations:                  175
Model:                          Logit   Df Residuals:                      152
Method:                           MLE   Df Model:                           22
Date:                Wed, 22 May 2024   Pseudo R-squ.:                  0.5976
Time:                        09:17:08   Log-Likelihood:                -39.266
converged:                       True   LL-Null:                       -97.576
Covariance Type:            nonrobust   LLR p-value:                 7.140e-15
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
D2                   2.8218      3.448      0.818      0.413      -3.936       9.580
DFA                  1.4404      2.320      0.621      0.535      -3.107       5.987
HNR                  3.0901      6.002      0.515      0.607      -8.674      14.854
Jitter:DDP          45.6740   2956.389      0.015      0.988   -5748.741    5840.089
MDVP:APQ            31.0449     53.564      0.580      0.562     -73.939     136.029
MDVP:Fhi(Hz)        -1.2380      1.961     -0.631      0.528      -5.082       2.606
MDVP:Flo(Hz)         1.0740      2.186      0.491      0.623      -3.211       5.359
MDVP:Fo(Hz)         -4.5819      3.936     -1.164      0.244     -12.297       3.133
MDVP:Jitter(%)     -39.0447     40.449     -0.965      0.334    -118.323      40.233
MDVP:Jitter(Abs)   -14.3703     23.721     -0.606      0.545     -60.862      32.122
MDVP:PPQ           -55.8013     38.159     -1.462      0.144    -130.592      18.989
MDVP:RAP            45.8953   2955.587      0.016      0.988   -5746.948    5838.739
MDVP:Shimmer       -40.6224    119.684     -0.339      0.734    -275.199     193.954
MDVP:Shimmer(dB)    74.3763     53.297      1.395      0.163     -30.085     178.837
NHR                  9.2313     17.548      0.526      0.599     -25.163      43.625
PPE                 18.3958     13.277      1.386      0.166      -7.627      44.418
RPDE                -1.9728      2.192     -0.900      0.368      -6.270       2.324
Shimmer:APQ3       -10.7878   6499.355     -0.002      0.999   -1.27e+04    1.27e+04
Shimmer:APQ5         5.6487     36.853      0.153      0.878     -66.582      77.880
Shimmer:DDA        -12.3804   6500.456     -0.002      0.998   -1.28e+04    1.27e+04
const               -6.7595      5.692     -1.187      0.235     -17.916       4.397
spread1              0.4163     10.613      0.039      0.969     -20.384      21.217
spread2              4.8739      2.833      1.721      0.085      -0.678      10.426
====================================================================================

Possibly complete quasi-separation: A fraction 0.17 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [17]:
def cut_off(y,threshold):
    Y = y.copy()
    Y[Y>threshold] = 1
    Y[Y<=threshold] = 0
    return(Y.astype(int))

In [18]:
test_y_pred_prob = results.predict(test_x)
test_y_pred = cut_off(test_y_pred_prob, 0.8)

In [19]:
f1_score(test_y, test_y_pred)

0.7857142857142857